In [7]:
import sys
print(sys.executable)

d:\projects\doc-intelligence-rag\.venv\Scripts\python.exe


In [9]:
# RAG System from First Principles
# =====================================
# This notebook builds a RAG system step-by-step
# We'll understand WHY before we code
#
# Prerequisites with UV:
#   uv init rag-learning
#   uv add jupyter numpy pandas requests pydantic groq
#   uv venv && source .venv/bin/activate
#
# Free APIs Used:
#   - Groq (LLM): https://console.groq.com (free API key)
#   - Ollama (Embeddings): https://ollama.ai (local, completely free)

# ========== PART 0: THE PROBLEM ==========
# 
# Problem: How do we make an LLM answer questions about OUR documents?
# 
# Naive approach: Put entire document into prompt
#   ❌ Problem: Context window is limited (4k, 8k, 128k tokens)
#   ❌ Problem: Costs scale with document size
#   ❌ Problem: LLM gets confused with irrelevant information
#
# Better approach: RAG (Retrieval-Augmented Generation)
#   ✓ Only pass relevant chunks to LLM
#   ✓ Reduces costs
#   ✓ Improves accuracy
#
# RAG Pipeline:
#   1. Split document into chunks
#   2. Convert chunks to embeddings (vectors)
#   3. Store embeddings in vector database
#   4. When user asks question:
#      a) Convert question to embedding
#      b) Find most similar chunks (similarity search)
#      c) Pass those chunks + question to LLM
#      d) LLM answers based on those chunks

print("=" * 60)
print("PART 0: RAG Pipeline Overview")
print("=" * 60)
print("""
Input: User asks "What is machine learning?"
       
       ↓
       
1. RETRIEVE: Search similar docs
   Question embedding → Find top 3 most similar chunks
   
2. AUGMENT: Build context
   Combine retrieved chunks into a context string
   
3. GENERATE: LLM answers
   Pass (context + question) → LLM → Answer

Output: "Machine learning is..."
""")

# ========== PART 1: TEXT CHUNKING ==========
print("\n" + "=" * 60)
print("PART 1: Text Chunking (Why?)")
print("=" * 60)

print("""
WHY chunk text?
  • Embeddings work better on ~500 token chunks (not 50k token documents)
  • Allows granular retrieval (retrieve only relevant section)
  • Reduces embedding costs

CHALLENGE: Chunking strategy matters!
  • Too small: Lose context (100 tokens)
  • Too large: Lose precision (5000 tokens)
  • SOLUTION: Use overlap (e.g., 500 token chunk with 50 token overlap)

WHY overlap?
  • Important info might be at chunk boundary
  • Overlap ensures semantic continuity
  • Example: "The study shows A. B supports A." might split badly without overlap
""")


PART 0: RAG Pipeline Overview

Input: User asks "What is machine learning?"

       ↓

1. RETRIEVE: Search similar docs
   Question embedding → Find top 3 most similar chunks

2. AUGMENT: Build context
   Combine retrieved chunks into a context string

3. GENERATE: LLM answers
   Pass (context + question) → LLM → Answer

Output: "Machine learning is..."


PART 1: Text Chunking (Why?)

WHY chunk text?
  • Embeddings work better on ~500 token chunks (not 50k token documents)
  • Allows granular retrieval (retrieve only relevant section)
  • Reduces embedding costs

CHALLENGE: Chunking strategy matters!
  • Too small: Lose context (100 tokens)
  • Too large: Lose precision (5000 tokens)
  • SOLUTION: Use overlap (e.g., 500 token chunk with 50 token overlap)

WHY overlap?
  • Important info might be at chunk boundary
  • Overlap ensures semantic continuity
  • Example: "The study shows A. B supports A." might split badly without overlap



In [13]:

# Implement chunking
def chunk_text(text, chunk_size=500, overlap=50):
    """
    Split text into overlapping chunks.
    
    Args:
        text: Raw text to chunk
        chunk_size: Tokens per chunk (roughly words * 0.75)
        overlap: Tokens overlap between chunks
    
    Returns:
        List of chunk dicts with text and metadata
    """
    words = text.split()
    chunks = []
    
    # Calculate stride (how many words to move forward each iteration)
    # If overlap=50 and chunk_size=500, stride=450
    stride = chunk_size - overlap
    
    for i in range(0, len(words), stride):
        chunk_words = words[i:i + chunk_size]
        chunk_text = " ".join(chunk_words)
        
        if chunk_text.strip():  # Skip empty chunks
            chunks.append({
                "text": chunk_text,
                "start_idx": i,
                "word_count": len(chunk_words),
                "chunk_id": len(chunks)  # Simple ID
            })
    
    return chunks

# Test chunking
sample_text = """
Machine learning is a subset of artificial intelligence. 
It focuses on algorithms that learn from data. 
Deep learning uses neural networks with multiple layers. 
Transformers are the backbone of modern NLP systems. 
The attention mechanism allows models to focus on relevant parts.
"""

chunks = chunk_text(sample_text, chunk_size=20, overlap=10)
print(f"\n✓ Split into {len(chunks)} chunks:")
for i, chunk in enumerate(chunks):
    print(f"  Chunk {i}: {chunk['word_count']} words | {chunk['text'][:60]}...")



✓ Split into 5 chunks:
  Chunk 0: 20 words | Machine learning is a subset of artificial intelligence. It ...
  Chunk 1: 20 words | on algorithms that learn from data. Deep learning uses neura...
  Chunk 2: 20 words | networks with multiple layers. Transformers are the backbone...
  Chunk 3: 12 words | NLP systems. The attention mechanism allows models to focus ...
  Chunk 4: 2 words | relevant parts....


In [14]:

# ========== PART 2: EMBEDDINGS ==========
print("\n" + "=" * 60)
print("PART 2: Embeddings (Why?)")
print("=" * 60)

print("""
PROBLEM: How do we compare text for similarity?
  • Can't just use string matching ("cat" ≠ "feline")
  • Need semantic understanding

SOLUTION: Embeddings (vectors that capture meaning)
  • Convert text → vector (list of numbers)
  • Similar texts have similar vectors
  • We can use math to compare them!

EXAMPLE:
  "The cat sat on the mat" → [0.2, -0.5, 0.8, 0.1, ...] (384 dims)
  "A feline sat on rug"    → [0.21, -0.48, 0.79, 0.12, ...] (384 dims)
  
  Notice: Very similar vectors = similar meaning!

HOW DO WE GET EMBEDDINGS?
  Option 1: Use OpenAI API (costs money)
  Option 2: Use local Ollama (free, slower)
  
  We'll use Ollama because:
    ✓ Free (no API costs)
    ✓ Privacy (stays on your machine)
    ✓ Fast for this use case
    ✓ Good enough for RAG

EMBEDDING MODELS:
  • nomic-embed-text: 384 dimensions (good for RAG)
  • all-minilm-l6-v2: 384 dimensions (lighter)
  • openai embedding: 1536 dimensions (more expensive, higher quality)
""")



PART 2: Embeddings (Why?)

PROBLEM: How do we compare text for similarity?
  • Can't just use string matching ("cat" ≠ "feline")
  • Need semantic understanding

SOLUTION: Embeddings (vectors that capture meaning)
  • Convert text → vector (list of numbers)
  • Similar texts have similar vectors
  • We can use math to compare them!

EXAMPLE:
  "The cat sat on the mat" → [0.2, -0.5, 0.8, 0.1, ...] (384 dims)
  "A feline sat on rug"    → [0.21, -0.48, 0.79, 0.12, ...] (384 dims)

  Notice: Very similar vectors = similar meaning!

HOW DO WE GET EMBEDDINGS?
  Option 1: Use OpenAI API (costs money)
  Option 2: Use local Ollama (free, slower)

  We'll use Ollama because:
    ✓ Free (no API costs)
    ✓ Privacy (stays on your machine)
    ✓ Fast for this use case
    ✓ Good enough for RAG

EMBEDDING MODELS:
  • nomic-embed-text: 384 dimensions (good for RAG)
  • all-minilm-l6-v2: 384 dimensions (lighter)
  • openai embedding: 1536 dimensions (more expensive, higher quality)



In [22]:

import numpy as np

def simulate_embedding(text, dims=348):
    """
    Simulate what an embedding looks like.
    
    In reality, Ollama would use a neural network to create this.
    For learning, we'll just use a hash-based deterministic vector.
    """
    # Hash the text to get consistent numbers
    hash_val = hash(text) % 10000
    np.random.seed(hash_val)
    
    # Create random but consistent vector
    embedding = np.random.randn(dims)
    
    # Normalize to unit length (important for cosine similarity)
    embedding = embedding / np.linalg.norm(embedding)
    
    return embedding.tolist()

# Demonstrate
text1 = "Machine learning is AI"
text2 = "Deep learning uses neural networks"
text3 = "Cooking pasta is delicious"

emb1 = simulate_embedding(text1)
emb2 = simulate_embedding(text2)
emb3 = simulate_embedding(text3)

print(f"\n✓ Embedding shape: {len(emb1)} dimensions")
print(f"  Sample values: {emb1[:5]} ...")



✓ Embedding shape: 348 dimensions
  Sample values: [-0.019382589036581188, -0.005570373852637493, -0.05710695701024572, -0.04601154504073093, 0.08037614551187885] ...


In [28]:

# ========== PART 3: SIMILARITY SEARCH ==========
print("\n" + "=" * 60)
print("PART 3: Similarity Search (Why?)")
print("=" * 60)

print("""
GOAL: Find which chunks are most relevant to a query

METHOD: Cosine Similarity
  • Measure angle between vectors
  • Values from -1 to 1 (1 = identical direction = same meaning)
  • Formula: similarity = (A · B) / (|A| * |B|)
  
EXAMPLE:
  Query: "What is deep learning?"
  Chunk 1: "Deep learning uses neural networks" → similarity = 0.92 ✓ Relevant!
  Chunk 2: "Cooking pasta..." → similarity = 0.15 ✗ Not relevant
  Chunk 3: "Neural networks have many layers" → similarity = 0.85 ✓ Relevant!
  
  Return top 2: Chunks 1 and 3
""")

def cosine_similarity(vec_a, vec_b):
    """
    Calculate cosine similarity between two vectors.
    
    Returns value between -1 and 1 (higher = more similar)
    """
    a = np.array(vec_a)
    b = np.array(vec_b)
    
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    return float(dot_product / (norm_a * norm_b))

# Test similarity
query = "neural networks"
query_emb = simulate_embedding(query)

similarities = [
    ("Deep learning uses neural networks", cosine_similarity(query_emb, emb2)),
    ("Machine learning is AI", cosine_similarity(query_emb, emb1)),
    ("Cooking pasta is delicious", cosine_similarity(query_emb, emb3)),
]

# Sort by similarity
similarities.sort(key=lambda x: x[1], reverse=False)

print(f"\n✓ Query: '{query}'")
print(f"  Results (sorted by relevance):")
for text, score in similarities:
    bar = "█" * int(score * 20)
    print(f"    {bar} {score:.3f} | {text}")



PART 3: Similarity Search (Why?)

GOAL: Find which chunks are most relevant to a query

METHOD: Cosine Similarity
  • Measure angle between vectors
  • Values from -1 to 1 (1 = identical direction = same meaning)
  • Formula: similarity = (A · B) / (|A| * |B|)

EXAMPLE:
  Query: "What is deep learning?"
  Chunk 1: "Deep learning uses neural networks" → similarity = 0.92 ✓ Relevant!
  Chunk 2: "Cooking pasta..." → similarity = 0.15 ✗ Not relevant
  Chunk 3: "Neural networks have many layers" → similarity = 0.85 ✓ Relevant!

  Return top 2: Chunks 1 and 3


✓ Query: 'neural networks'
  Results (sorted by relevance):
     -0.077 | Deep learning uses neural networks
     -0.021 | Cooking pasta is delicious
     -0.005 | Machine learning is AI


In [29]:

# ========== PART 4: VECTOR STORE ==========
print("\n" + "=" * 60)
print("PART 4: Vector Store (Why?)")
print("=" * 60)

print("""
PROBLEM: How do we store and retrieve embeddings efficiently?

SIMPLE APPROACH: In-memory dictionary
  ✓ Easy to understand and implement
  ✓ Fast for small documents (< 10k chunks)
  ✗ Loses data when program stops
  ✗ Doesn't scale to millions of vectors

PRODUCTION APPROACH: Vector databases
  • Pinecone, Weaviate, Milvus, Chroma, Qdrant
  • Optimized for similarity search
  • Persistent storage
  • Scales to billions of vectors

FOR NOW: We'll use in-memory (simple) but structure it to be replaceable
""")

class SimpleVectorStore:
    """
    In-memory vector store.
    Structure: Can easily swap for Pinecone/Weaviate later.
    """
    
    def __init__(self):
        self.vectors = {}      # chunk_id -> embedding vector
        self.metadata = {}     # chunk_id -> {text, source, etc}
    
    def add(self, chunk_id, text, embedding):
        """Store a chunk with its embedding."""
        self.vectors[chunk_id] = embedding
        self.metadata[chunk_id] = {"text": text, "length": len(text)}
    
    def search(self, query_embedding, top_k=3):
        """Find most similar chunks."""
        if not self.vectors:
            return []
        
        results = []
        for chunk_id, vector in self.vectors.items():
            similarity = cosine_similarity(query_embedding, vector)
            results.append({
                "chunk_id": chunk_id,
                "similarity": similarity,
                "text": self.metadata[chunk_id]["text"]
            })
        
        # Sort by similarity descending
        results.sort(key=lambda x: x["similarity"], reverse=True)
        return results[:top_k]

# Test vector store
store = SimpleVectorStore()

# Add chunks with embeddings
for chunk in chunks:
    emb = simulate_embedding(chunk["text"])
    store.add(chunk["chunk_id"], chunk["text"], emb)

# Search
query = "transformers attention mechanism"
query_emb = simulate_embedding(query)
results = store.search(query_emb, top_k=2)

print(f"\n✓ Query: '{query}'")
print(f"  Retrieved {len(results)} chunks:")
for r in results:
    print(f"    [{r['similarity']:.3f}] {r['text'][:70]}...")



PART 4: Vector Store (Why?)

PROBLEM: How do we store and retrieve embeddings efficiently?

SIMPLE APPROACH: In-memory dictionary
  ✓ Easy to understand and implement
  ✓ Fast for small documents (< 10k chunks)
  ✗ Loses data when program stops
  ✗ Doesn't scale to millions of vectors

PRODUCTION APPROACH: Vector databases
  • Pinecone, Weaviate, Milvus, Chroma, Qdrant
  • Optimized for similarity search
  • Persistent storage
  • Scales to billions of vectors

FOR NOW: We'll use in-memory (simple) but structure it to be replaceable


✓ Query: 'transformers attention mechanism'
  Retrieved 2 chunks:
    [0.005] on algorithms that learn from data. Deep learning uses neural networks...
    [-0.012] NLP systems. The attention mechanism allows models to focus on relevan...


In [31]:

# ========== PART 5: LLM INTEGRATION ==========
print("\n" + "=" * 60)
print("PART 5: LLM Integration (Why?)")
print("=" * 60)

print("""
PROBLEM: How do we actually answer the user's question?

SOLUTION: Pass context + question to LLM
  1. Retrieve relevant chunks (from vector store)
  2. Combine into "context" string
  3. Create prompt: "Context: [chunks] Question: [query]"
  4. Send to LLM (Groq, OpenAI, etc)
  5. LLM answers based on context

WHY THIS WORKS:
  • LLM has knowledge from training
  • Context grounds answer in YOUR documents
  • LLM won't hallucinate (ideally) because answer must match context

EXAMPLE PROMPT:
  ---
  Context:
  Deep learning uses neural networks with many layers.
  The attention mechanism focuses on relevant parts.
  
  Question: How does deep learning work?
  
  Answer: (LLM fills this in)
  ---
""")

def build_context(retrieved_chunks):
    """
    Combine retrieved chunks into a context string for the LLM.
    
    This is where you control what the LLM sees.
    """
    context = ""
    for i, chunk in enumerate(retrieved_chunks):
        score = chunk.get("similarity", 0)
        context += f"[Chunk {i+1} - Relevance: {score:.1%}]\n"
        context += chunk["text"] + "\n\n"
    
    return context

# Demo
context = build_context(results)
print(f"\n✓ Built context from {len(results)} chunks:")
print(f"  Length: {len(context)} characters")
print(f"  Preview:\n  {context[:200]}...")





PART 5: LLM Integration (Why?)

PROBLEM: How do we actually answer the user's question?

SOLUTION: Pass context + question to LLM
  1. Retrieve relevant chunks (from vector store)
  2. Combine into "context" string
  3. Create prompt: "Context: [chunks] Question: [query]"
  4. Send to LLM (Groq, OpenAI, etc)
  5. LLM answers based on context

WHY THIS WORKS:
  • LLM has knowledge from training
  • Context grounds answer in YOUR documents
  • LLM won't hallucinate (ideally) because answer must match context

EXAMPLE PROMPT:
  ---
  Context:
  Deep learning uses neural networks with many layers.
  The attention mechanism focuses on relevant parts.

  Question: How does deep learning work?

  Answer: (LLM fills this in)
  ---


✓ Built context from 2 chunks:
  Length: 271 characters
  Preview:
  [Chunk 1 - Relevance: 0.5%]
on algorithms that learn from data. Deep learning uses neural networks with multiple layers. Transformers are the backbone of modern

[Chunk 2 - Relevance: -1.2%]
NLP 

In [34]:
def build_prompt(context, query):
    """Build final prompt for LLM."""
    prompt = f"""You are a helpful assistant. Answer the question based ONLY on the provided context.
If the context doesn't contain the answer, say so explicitly.

Context:
{context}

Question: {query}

Answer:"""
    return prompt
    
# ========== PART 6: PUTTING IT TOGETHER ==========
print("\n" + "=" * 60)
print("PART 6: Complete RAG Pipeline")
print("=" * 60)

def rag_pipeline(query, document_text, chunk_size=500, overlap=50, top_k=3):
    """
    Complete RAG pipeline:
    1. Chunk document
    2. Create embeddings
    3. Build vector store
    4. Retrieve relevant chunks
    5. Build prompt
    6. Return to user (in real system, send to LLM)
    """
    
    print(f"\n[STEP 1] Chunking document...")
    chunks = chunk_text(document_text, chunk_size, overlap)
    print(f"  → Split into {len(chunks)} chunks")
    
    print(f"\n[STEP 2] Creating embeddings...")
    store = SimpleVectorStore()
    for chunk in chunks:
        emb = simulate_embedding(chunk["text"])
        store.add(chunk["chunk_id"], chunk["text"], emb)
    print(f"  → Created {len(chunks)} embeddings")
    
    print(f"\n[STEP 3] Embedding query...")
    query_emb = simulate_embedding(query)
    print(f"  → Query embedding ready")
    
    print(f"\n[STEP 4] Searching similar chunks...")
    retrieved = store.search(query_emb, top_k)
    print(f"  → Retrieved {len(retrieved)} chunks")
    
    print(f"\n[STEP 5] Building prompt...")
    context = build_context(retrieved)
    prompt = build_prompt(context, query)
    print(f"  → Prompt ready ({len(prompt)} chars)")
    
    return {
        "query": query,
        "chunks_created": len(chunks),
        "chunks_retrieved": len(retrieved),
        "context": context,
        "prompt": prompt,
        "retrieved_chunks": retrieved
    }

# Test full pipeline
doc = """
Machine learning is a subset of AI that learns from data.
Deep learning uses neural networks with many layers.
Transformers use attention mechanisms for NLP tasks.
The attention mechanism allows models to focus on important parts.
Large language models are trained on massive datasets.
"""

result = rag_pipeline("How do transformers work?", doc, chunk_size=500, top_k=2)

print(f"\n" + "=" * 60)
print("RESULT SUMMARY")
print("=" * 60)
print(f"Query: {result['query']}")
print(f"Chunks created: {result['chunks_created']}")
print(f"Chunks retrieved: {result['chunks_retrieved']}")
print(f"\nTop retrieved chunks:")
for chunk in result['retrieved_chunks']:
    print(f"  [{chunk['similarity']:.2%}] {chunk['text'][:60]}...")
print(f"\nFinal prompt (to be sent to LLM):")
print(result['prompt'])

print("\n" + "=" * 60)
print("✓ You now understand RAG!")
print("=" * 60)
print("""
Next steps:
  1. Replace simulate_embedding() with real Ollama calls
  2. Replace our Vector Store with Pinecone/Chroma
  3. Replace the final prompt with real Groq LLM calls
  4. Wrap in FastAPI for production
  
But the LOGIC stays the same!
""")


PART 6: Complete RAG Pipeline

[STEP 1] Chunking document...
  → Split into 1 chunks

[STEP 2] Creating embeddings...
  → Created 1 embeddings

[STEP 3] Embedding query...
  → Query embedding ready

[STEP 4] Searching similar chunks...
  → Retrieved 1 chunks

[STEP 5] Building prompt...
  → Prompt ready (518 chars)

RESULT SUMMARY
Query: How do transformers work?
Chunks created: 1
Chunks retrieved: 1

Top retrieved chunks:
  [6.46%] Machine learning is a subset of AI that learns from data. De...

Final prompt (to be sent to LLM):
You are a helpful assistant. Answer the question based ONLY on the provided context.
If the context doesn't contain the answer, say so explicitly.

Context:
[Chunk 1 - Relevance: 6.5%]
Machine learning is a subset of AI that learns from data. Deep learning uses neural networks with many layers. Transformers use attention mechanisms for NLP tasks. The attention mechanism allows models to focus on important parts. Large language models are trained on massive da

In [38]:
from groq import Groq
import os

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

models = client.models.list()
for m in models.data:
    print(m.id)

llama-3.1-8b-instant
groq/compound-mini
whisper-large-v3
moonshotai/kimi-k2-instruct-0905
openai/gpt-oss-20b
playai-tts-arabic
groq/compound
whisper-large-v3-turbo
openai/gpt-oss-120b
meta-llama/llama-prompt-guard-2-86m
meta-llama/llama-prompt-guard-2-22m
openai/gpt-oss-safeguard-20b
moonshotai/kimi-k2-instruct
qwen/qwen3-32b
meta-llama/llama-4-maverick-17b-128e-instruct
allam-2-7b
meta-llama/llama-guard-4-12b
playai-tts
meta-llama/llama-4-scout-17b-16e-instruct
llama-3.3-70b-versatile


In [39]:
from dotenv import load_dotenv
import os
from groq import Groq

# Load from .env file
load_dotenv()

# Get API key safely
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    raise ValueError("GROQ_API_KEY not found in environment or .env file")

def query_groq(context, query):
    """Get real answer from Groq LLM."""
    client = Groq(api_key=api_key)
    
    message = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        max_tokens=1024,
        messages=[{
            "role": "user",
            "content": f"""Based on this context, answer the question.
Only use the context provided.

Context:
{context}

Question: {query}

Answer:"""
        }]
    )
    return message.choices[0].message.content

# Test it
answer = query_groq(result["context"], result["query"])
print(f"\n🤖 LLM ANSWER:\n{answer}")


🤖 LLM ANSWER:
According to the provided context, transformers use attention mechanisms for NLP tasks.
